In [1]:
from sym_so3 import *
from drake_sympy import *
import debug

Using the standard formulas:

Let $r = [r_r, r_p, r_y]$ be the Euler angles, and look at derivative values

\begin{align}
    \text{Skew symmetric angular velocity matrix:} \\
    \hat{\omega} &= R^T \dot{R}   \\
    \text{Skew symmetric angular acceleration matrix:} \\
    \hat{\alpha} &= \dot{R}^T \dot{R} + R^T \ddot{R} \\
        &= \hat{\omega}^T \hat{\omega} + R^T \ddot{R} \\
    \text{Alternatively, can use time derivative of (unskewed) angular velocity} \\
    \alpha &= \dot{\omega} = \frac{\partial \omega}{\partial r} \dot{r} + \frac{\partial \omega}{\partial \dot{r}} \ddot{r}
\end{align}

In [2]:
from pydrake.symbolic import Jacobian

In [3]:
workspace = SecondOrderWorkspace.make(["r", "p", "y"])

r_s, rd_s, rdd_s = workspace.syms

r_rd_s = cat(r_s, rd_s)
rd_rdd_s = cat(rd_s, rdd_s)

# Compute rotation matrix using rpy.
R_s = RollPitchYaw_[Expression](r_s).ToRotationMatrix().matrix()

# Compute first- and second-derivatives w.r.t. time.
Rd_s = derivatives_1st(R_s, r_s, rd_s)
Rd_s, Rdd_s = derivatives_1st_and_2nd(R_s, r_s, rd_s, rdd_s)
# Compute skew-symmetric angular velocity matrix.
wh_s = R_s.T @ Rd_s
w_s = unskew(wh_s, tol=None)

# Compute angular accel. directly from angular velocity.
ah_s = derivatives_1st(wh_s, r_rd_s, rd_rdd_s)

# Alternative
ah_alt_s = wh_s.T @ wh_s + R_s.T @ Rdd_s

In [4]:
J_s = Jacobian(w_s, rd_s)

In [5]:
env = workspace.env
r_v, rd_v, rdd_v = workspace.values

# # Single-axis: This shows numerical values that are OK for both angular accelerations.
# r_v[:] = [0.0, 0.0, 0.4]
# rd_v[:] = [0.0, 0.0, 0.7]
# rdd_v[:] = [0.0, 0.0, 1.1]

# Multi-axis: This does *not* show good numerical values, which is reflected in symbolics.
r_v[:] = [0.2, 0.3, 0.4]
rd_v[:] = [0.5, 0.6, 0.7]
rdd_v[:] = [0.8, 0.9, 1.1]

R = sym.Evaluate(R_s, env)
wh = sym.Evaluate(wh_s, env)
w = unskew(wh)

ah = sym.Evaluate(ah_s, env)
a = unskew(ah)
ah_alt = sym.Evaluate(ah_alt_s, env)
a_alt = unskew(ah_alt)

In [6]:
# Decent-looking values.
print(R @ R.T)
print(w)
print(a)
print(a_alt)

[[ 1.00000000e+00 -5.54075129e-17  6.97071887e-17]
 [-5.54075129e-17  1.00000000e+00  2.18104550e-17]
 [ 6.97071887e-17  2.18104550e-17  1.00000000e+00]]
[0.29313586 0.72089719 0.53620376]
[0.07368645 1.33427893 0.36902733]
[0.07368645 1.33427893 0.36902733]


In [7]:
to_sympy = make_drake_to_sympy(cat(r_s, rd_s, rdd_s))
r_sympy = [to_sympy[hash(x)] for x in r_s]

def pretty(A, *, simplify=True):
    A = drake_to_sympy_matrix(A, to_sympy)
    A = pretty_trig(A, r_sympy, simplify=simplify)
    return A

The `trigsimp` stuff will take a few min

For compactness, I substitute stuff like $cos(r)$ with $c_r$, $sin(y)$ with $s_y$, etc.

In [8]:
pretty(R_s)

Matrix([
[c_p*c_y, -c_r*s_y + c_y*s_p*s_r, c_r*c_y*s_p + s_r*s_y],
[c_p*s_y,  c_r*c_y + s_p*s_r*s_y, c_r*s_p*s_y - c_y*s_r],
[   -s_p,                c_p*s_r,               c_p*c_r]])

In [9]:
pretty(Rd_s)

Matrix([
[-c_p*s_y*ydot - c_y*pdot*s_p,  c_p*c_y*pdot*s_r + rdot*(c_r*c_y*s_p + s_r*s_y) - ydot*(c_r*c_y + s_p*s_r*s_y), c_p*c_r*c_y*pdot + rdot*(c_r*s_y - c_y*s_p*s_r) + ydot*(-c_r*s_p*s_y + c_y*s_r)],
[ c_p*c_y*ydot - pdot*s_p*s_y, c_p*pdot*s_r*s_y + rdot*(c_r*s_p*s_y - c_y*s_r) + ydot*(-c_r*s_y + c_y*s_p*s_r),  c_p*c_r*pdot*s_y - rdot*(c_r*c_y + s_p*s_r*s_y) + ydot*(c_r*c_y*s_p + s_r*s_y)],
[                   -c_p*pdot,                                                     c_p*c_r*rdot - pdot*s_p*s_r,                                                    -c_p*rdot*s_r - c_r*pdot*s_p]])

In [10]:
pretty(w_s)

Matrix([
[        rdot - s_p*ydot],
[c_p*s_r*ydot + c_r*pdot],
[c_p*c_r*ydot - pdot*s_r]])

In [11]:
pretty(J_s)

Matrix([
[1,    0,    -s_p],
[0,  c_r, c_p*s_r],
[0, -s_r, c_p*c_r]])

In [12]:
pretty(Rdd_s)

Matrix([
[-c_p*c_y*pdot**2 - c_p*c_y*ydot**2 - c_p*s_y*yddot - c_y*pddot*s_p + 2*pdot*s_p*s_y*ydot, 2*c_p*c_r*c_y*pdot*rdot + c_p*c_y*pddot*s_r - 2*c_p*pdot*s_r*s_y*ydot + c_r*c_y*rddot*s_p - c_r*c_y*yddot + c_r*rdot**2*s_y - 2*c_r*rdot*s_p*s_y*ydot + c_r*s_y*ydot**2 - c_y*pdot**2*s_p*s_r - c_y*rdot**2*s_p*s_r + 2*c_y*rdot*s_r*ydot - c_y*s_p*s_r*ydot**2 + rddot*s_r*s_y - s_p*s_r*s_y*yddot, c_p*c_r*c_y*pddot - 2*c_p*c_r*pdot*s_y*ydot - 2*c_p*c_y*pdot*rdot*s_r - c_r*c_y*pdot**2*s_p - c_r*c_y*rdot**2*s_p + 2*c_r*c_y*rdot*ydot - c_r*c_y*s_p*ydot**2 + c_r*rddot*s_y - c_r*s_p*s_y*yddot - c_y*rddot*s_p*s_r + c_y*s_r*yddot - rdot**2*s_r*s_y + 2*rdot*s_p*s_r*s_y*ydot - s_r*s_y*ydot**2],
[ c_p*c_y*yddot - c_p*pdot**2*s_y - c_p*s_y*ydot**2 - 2*c_y*pdot*s_p*ydot - pddot*s_p*s_y, 2*c_p*c_r*pdot*rdot*s_y + 2*c_p*c_y*pdot*s_r*ydot + c_p*pddot*s_r*s_y - c_r*c_y*rdot**2 + 2*c_r*c_y*rdot*s_p*ydot - c_r*c_y*ydot**2 + c_r*rddot*s_p*s_y - c_r*s_y*yddot - c_y*rddot*s_r + c_y*s_p*s_r*yddot - pdot**2*s_p*s_r*

In [13]:
pretty(ah_s)

Matrix([
[                                                                                 0, -c_p*c_r*yddot + c_r*pdot*s_p*ydot + pddot*s_r + rdot*(c_p*s_r*ydot + c_r*pdot), c_p*s_r*yddot + c_r*pddot - pdot*s_p*s_r*ydot + rdot*(c_p*c_r*ydot - pdot*s_r)],
[c_p*c_r*yddot - 1.0*c_r*pdot*s_p*ydot - pddot*s_r - rdot*(c_p*s_r*ydot + c_r*pdot),                                                                               0,                                              c_p*pdot*ydot - rddot + s_p*yddot],
[  -c_p*s_r*yddot - c_r*pddot + pdot*s_p*s_r*ydot + rdot*(-c_p*c_r*ydot + pdot*s_r),                                              -c_p*pdot*ydot + rddot - s_p*yddot,                                                                              0]])

In [14]:
pretty(ah_alt_s)

Matrix([
[                                                                                 0, -c_p*c_r*yddot + c_p*rdot*s_r*ydot + c_r*pdot*rdot + c_r*pdot*s_p*ydot + pddot*s_r, c_p*c_r*rdot*ydot + c_p*s_r*yddot + c_r*pddot - pdot*rdot*s_r - pdot*s_p*s_r*ydot],
[ c_p*c_r*yddot - c_p*rdot*s_r*ydot - c_r*pdot*rdot - c_r*pdot*s_p*ydot - pddot*s_r,                                                                                  0,                                                 c_p*pdot*ydot - rddot + s_p*yddot],
[-c_p*c_r*rdot*ydot - c_p*s_r*yddot - c_r*pddot + pdot*rdot*s_r + pdot*s_p*s_r*ydot,                                                 -c_p*pdot*ydot + rddot - s_p*yddot,                                                                                 0]])